In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import optuna
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

c:\Users\Shrinivass\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Syntetically Generated Dataset

In [2]:
# 1. Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
X_train.shape

(800, 20)

In [4]:
X_train[0]

array([ 0.50363664, -1.51368248, -0.46907062,  1.90176571, -0.87064279,
        1.82004715,  1.66291365,  1.29105223, -0.16713608, -1.04718436,
        1.43003039,  0.20104766,  1.27577182, -1.13260729,  1.75008532,
       -1.4089039 ,  0.03301588, -0.80340946, -1.31410638,  1.41209637])

In [5]:
y_train[0]

1

In [6]:
# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)

### Defining a Neural Network

In [9]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)  # Output layer for binary classification
        )
        
    def forward(self, x):
        return self.network(x)

### Performing Hyperparameter Tunning

In [ ]:
# 3. Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)

    # Model, loss, optimizer
    model = SimpleNN(input_dim=20, hidden_dim=hidden_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    epochs = 20
    batch_size = 32
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Validation accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = correct / total
    return accuracy

# 4. Run the Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 5. Print the best hyperparameters
print("Best hyperparameters found:")
print(study.best_params)            

[I 2026-01-15 16:52:15,918] A new study created in memory with name: no-name-f635e0bc-a9cc-4f15-afb2-4489529e883d


C:\Users\Shrinivass\AppData\Local\Temp\ipykernel_22804\3446579323.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2026-01-15 16:52:20,919] Trial 0 finished with value: 0.825 and parameters: {'learning_rate': 0.00010434520036826442, 'hidden_dim': 90}. Best is trial 0 with value: 0.825.
C:\Users\Shrinivass\AppData\Local\Temp\ipykernel_22804\3446579323.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2026-01-15 16:52:21,789] Trial 1 finished with value: 0.815 and parameters: {'learning_rate': 0.01438102667985434, 'hidden_d

Best hyperparameters found:
{'learning_rate': 0.0003268230005518709, 'hidden_dim': 89}


In [20]:
# 1. Use the best params from Optuna
best_lr = 0.0003268230005518709
best_hidden_dim = 89

# 2. Setup final model, loss, and optimizer
model = SimpleNN(input_dim=20, hidden_dim=best_hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=best_lr)

# 3. Setup DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)

# 4. Final Training Loop
epochs = 50 
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/50, Loss: 0.6135
Epoch 2/50, Loss: 0.5661
Epoch 3/50, Loss: 0.5254
Epoch 4/50, Loss: 0.4908
Epoch 5/50, Loss: 0.4595
Epoch 6/50, Loss: 0.4328
Epoch 7/50, Loss: 0.4097
Epoch 8/50, Loss: 0.3899
Epoch 9/50, Loss: 0.3734
Epoch 10/50, Loss: 0.3589
Epoch 11/50, Loss: 0.3473
Epoch 12/50, Loss: 0.3372
Epoch 13/50, Loss: 0.3293
Epoch 14/50, Loss: 0.3219
Epoch 15/50, Loss: 0.3161
Epoch 16/50, Loss: 0.3106
Epoch 17/50, Loss: 0.3060
Epoch 18/50, Loss: 0.3022
Epoch 19/50, Loss: 0.2984
Epoch 20/50, Loss: 0.2952
Epoch 21/50, Loss: 0.2922
Epoch 22/50, Loss: 0.2897
Epoch 23/50, Loss: 0.2871
Epoch 24/50, Loss: 0.2851
Epoch 25/50, Loss: 0.2826
Epoch 26/50, Loss: 0.2803
Epoch 27/50, Loss: 0.2783
Epoch 28/50, Loss: 0.2770
Epoch 29/50, Loss: 0.2748
Epoch 30/50, Loss: 0.2734
Epoch 31/50, Loss: 0.2712
Epoch 32/50, Loss: 0.2695
Epoch 33/50, Loss: 0.2674
Epoch 34/50, Loss: 0.2657
Epoch 35/50, Loss: 0.2639
Epoch 36/50, Loss: 0.2623
Epoch 37/50, Loss: 0.2606
Epoch 38/50, Loss: 0.2590
Epoch 39/50, Loss: 0.